In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [80]:
#한글 처리를 위해 폰트 설정
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [81]:
df=pd.read_csv('C:/Users/tjoeun/Desktop/송수린/workspace/weather_project/data/예측용기상대기정보.csv')

In [82]:
df.head()

,지점명,일시,지점,기온(°C),강수여부,풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),해면기압(hPa),...,시정(10m),지면온도(°C),PM10,PM2.5,O3,NO2,CO,SO2,월,전날기온
0,서울,2023-02-24 17:00:00,108,7.0,0,3.5,290.0,31.0,1012.8,1023.4,...,2000.0,5.8,34.0,21.0,0.049,0.013,0.4,0.003,2,8.1
1,서울,2023-02-24 18:00:00,108,5.3,0,3.1,290.0,29.0,1013.4,1024.1,...,2000.0,2.6,31.0,20.0,0.048,0.012,0.3,0.003,2,6.3
2,서울,2023-02-24 19:00:00,108,3.8,0,2.7,270.0,34.0,1014.3,1025.0,...,2000.0,0.5,28.0,18.0,0.043,0.015,0.3,0.003,2,4.5
3,서울,2023-02-24 20:00:00,108,2.6,0,3.0,290.0,42.0,1015.3,1026.1,...,2000.0,-0.4,30.0,16.0,0.042,0.013,0.4,0.003,2,3.7
4,서울,2023-02-24 21:00:00,108,1.6,0,3.9,290.0,47.0,1015.6,1026.4,...,2000.0,-0.2,35.0,19.0,0.038,0.015,0.4,0.003,2,3.2


In [83]:
df=df[['지점명', '일시', '지점', '기온(°C)', '강수여부', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '적설(cm)', '전운량(10분위)', 'PM10',
       'PM2.5', 'O3', 'NO2', 'CO', 'SO2', '증기압(hPa)', '이슬점온도(°C)', '시정(10m)',
       '지면온도(°C)', '월', '전날기온']]

In [84]:
df

,지점명,일시,지점,기온(°C),강수여부,풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),해면기압(hPa),...,O3,NO2,CO,SO2,증기압(hPa),이슬점온도(°C),시정(10m),지면온도(°C),월,전날기온
0,서울,2023-02-24 17:00:00,108,7.0,0,3.5,290.0,31.0,1012.8,1023.4,...,0.049,0.013,0.4,0.003,3.1,-9.0,2000.0,5.8,2,8.1
1,서울,2023-02-24 18:00:00,108,5.3,0,3.1,290.0,29.0,1013.4,1024.1,...,0.048,0.012,0.3,0.003,2.6,-11.3,2000.0,2.6,2,6.3
2,서울,2023-02-24 19:00:00,108,3.8,0,2.7,270.0,34.0,1014.3,1025.0,...,0.043,0.015,0.3,0.003,2.7,-10.6,2000.0,0.5,2,4.5
3,서울,2023-02-24 20:00:00,108,2.6,0,3.0,290.0,42.0,1015.3,1026.1,...,0.042,0.013,0.4,0.003,3.1,-9.0,2000.0,-0.4,2,3.7
4,서울,2023-02-24 21:00:00,108,1.6,0,3.9,290.0,47.0,1015.6,1026.4,...,0.038,0.015,0.4,0.003,3.2,-8.5,2000.0,-0.2,2,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5976,순창군,2023-02-28 19:00:00,254,10.5,0,1.3,250.0,51.0,1009.4,1025.1,...,0.047,0.015,0.1,0.002,6.4,0.7,3392.0,6.9,2,9.4
5977,순창군,2023-02-28 20:00:00,254,9.3,0,1.2,250.0,58.0,1009.7,1025.5,...,0.042,0.014,0.1,0.001,6.8,1.4,3176.0,5.5,2,6.0
5978,순창군,2023-02-28 21:00:00,254,6.8,0,1.3,230.0,66.0,1009.8,1025.8,...,0.039,0.011,0.1,0.001,6.5,0.8,2701.0,4.2,2,4.2
5979,순창군,2023-02-28 22:00:00,254,5.3,0,1.1,200.0,74.0,1009.9,1026.0,...,0.026,0.014,0.1,0.001,6.6,1.0,2427.0,3.0,2,1.5


In [85]:
# 기온을 종속변수로 지정
y = df['기온(°C)'].copy()
X = df[df.columns[4:]].copy()

In [86]:
# 범위형 변수 제거
X.drop(columns=['풍향(16방위)', '전운량(10분위)', '월'], axis=1, inplace=True)

In [87]:
# 더미변수 생성
X=pd.get_dummies(data=X, columns=['강수여부'])

In [88]:
scaler = StandardScaler()
scaler.fit(X.iloc[:,:-2])
X_scaled = scaler.transform(X.iloc[:,:-2])
X_scaled = pd.concat([pd.DataFrame(X_scaled), X.iloc[:,-2:]], axis=1)
X_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,강수여부_0,강수여부_1
0,0.559461,-1.476411,0.027062,-0.410179,1.900782,-0.20727,-0.344580,-0.158106,1.087961,-0.191091,-0.186205,0.552630,-0.808163,-0.773893,-0.241920,0.343774,1.061467,1,0
1,0.353722,-1.572707,0.072989,-0.282906,1.650687,-0.20727,-0.488209,-0.219843,1.019802,-0.283920,-0.667491,0.552630,-1.050353,-1.166541,-0.241920,-0.185542,0.678861,1,0
2,0.147983,-1.331967,0.141880,-0.119269,-0.600170,-0.20727,-0.631837,-0.343318,0.679010,-0.005432,-0.667491,0.552630,-1.001915,-1.047039,-0.241920,-0.532906,0.296256,1,0
3,0.302288,-0.946782,0.218425,0.080732,-0.600170,-0.20727,-0.536085,-0.466794,0.610851,-0.191091,-0.186205,0.552630,-0.808163,-0.773893,-0.241920,-0.681777,0.126209,1,0
4,0.765200,-0.706041,0.241389,0.135277,-0.600170,-0.20727,-0.296704,-0.281581,0.338217,-0.005432,-0.186205,0.552630,-0.759725,-0.688535,-0.241920,-0.648694,0.019930,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5976,-0.572103,-0.513448,-0.233193,-0.101087,-0.600170,-0.20727,-0.679713,-0.096368,0.951644,-0.005432,-1.630061,-0.454463,0.790289,0.882057,0.742988,0.525727,1.337793,1,0
5977,-0.623538,-0.176412,-0.210229,-0.028360,-0.600170,-0.20727,-0.296704,0.027107,0.610851,-0.098262,-1.630061,-1.461557,0.984041,1.001559,0.590158,0.294151,0.615094,1,0
5978,-0.572103,0.208773,-0.202575,0.026186,-0.600170,-0.20727,-0.105199,0.088845,0.406376,-0.376750,-1.630061,-1.461557,0.838727,0.899129,0.254072,0.079116,0.232489,1,0
5979,-0.674972,0.593958,-0.194920,0.062550,-0.600170,-0.20727,-0.296704,0.088845,-0.479685,-0.098262,-1.630061,-1.461557,0.887165,0.933272,0.060204,-0.119378,-0.341419,1,0


In [89]:
X_scaled.columns = ['풍속(m/s)', '습도(%)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '적설(cm)', 'PM10', 'PM2.5', 
                    'O3', 'NO2', 'CO', 'SO2', '증기압(hPa)', '이슬점온도(°C)', '시정(10m)', '지면온도(°C)', '전날기온', 
                    '강수여부_0', '강수여부_1']

In [90]:
# 이슬점온도(°C) 제거
X_scaled.drop(columns=['이슬점온도(°C)'], axis=1, inplace=True)

In [91]:
# 증기압(hPa) 제거
X_scaled.drop(columns=['증기압(hPa)'], axis=1, inplace=True)

In [92]:
# 지면온도(°C) 제거
X_scaled.drop(columns=['지면온도(°C)'], axis=1, inplace=True)

In [93]:
# 상관계수가 높은 PM2.5 제거
X_scaled.drop(columns=['PM2.5'], axis=1, inplace=True)

In [94]:
# 해면기압(hPa) 제거
X_scaled.drop(columns=['해면기압(hPa)'], axis=1, inplace=True)

In [95]:
pre=X_scaled.copy()

In [75]:
t_X=pd.read_csv('학습용.csv')
t_Y=pd.read_csv('학습용y.csv')

In [76]:
X_train, X_test, y_train, y_test=train_test_split(t_X, t_Y, test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape)

(1353333, 14) (580000, 14)


In [49]:
%%time
#RandomizedSearchCV() 최근에 많이 사용되고 있는 파라미터 최적화 함수
#매개변수의 범위를 지정해주면 무작위로 매개변수를 조합하여 최적의 성능 측정
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs={
 'n_estimators': randint(low=1, high=100),
 'max_features': randint(low=1, high=8),
 }
forest_reg=RandomForestRegressor(random_state=0, n_jobs=-1)
rnd_search=RandomizedSearchCV(forest_reg, param_distributions=param_distribs, cv=5, random_state=0)
rnd_search.fit(X_train, y_train)

c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

Wall time: 1h 39min 46s


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=0),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000026339A201F0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000026339A20310>},
                   random_state=0)

In [50]:
cvres=rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(mean_score), params)

0.9606584552303622 {'max_features': 5, 'n_estimators': 48}
0.9605890525648031 {'max_features': 6, 'n_estimators': 65}
0.9613929692376384 {'max_features': 4, 'n_estimators': 68}
0.9606998128868366 {'max_features': 2, 'n_estimators': 84}
0.9599693595571622 {'max_features': 6, 'n_estimators': 37}
0.9605905127803541 {'max_features': 7, 'n_estimators': 89}
0.9488668199567677 {'max_features': 1, 'n_estimators': 13}
0.9614380074805443 {'max_features': 3, 'n_estimators': 66}
0.9597916917144983 {'max_features': 7, 'n_estimators': 40}
0.9605905127803541 {'max_features': 7, 'n_estimators': 89}


In [96]:
%%time
model_RSCV_forest=rnd_search.best_estimator_
y_pred=model_RSCV_forest.predict(X_scaled)
rms=np.sqrt(mean_squared_error(y, y_pred))
rms

Wall time: 108 ms


10.606906552578184

In [78]:
print('훈련용:',model_RSCV_forest.score(X_train, y_train))
print('검증용:',model_RSCV_forest.score(X_test, y_test))
print('예측용:',model_RSCV_forest.score(X_scaled, y))

훈련용: 0.989361125248774
검증용: 0.9262048954346926
예측용: -4.043011909570318


In [97]:
pd.DataFrame([y,y_pred])

,0,1,2,3,4,5,6,7,8,9,...,5971,5972,5973,5974,5975,5976,5977,5978,5979,5980
기온(°C),7.000000,5.30000,3.800000,2.60,1.600000,1.000000,0.400000,-0.400000,-0.800000,-1.600000,...,12.600000,13.600000,13.700000,14.100000,13.100000,10.50,9.300000,6.80000,5.300000,5.200000
Unnamed 0,25.239394,21.74697,15.810606,14.15,12.504545,10.486364,10.775758,9.542424,9.834848,10.556061,...,33.422727,33.201515,33.427273,33.572727,32.556061,25.35,17.478788,15.89697,8.333333,6.375758


array([25.23939394, 21.7469697 , 15.81060606, ..., 15.8969697 ,
        8.33333333,  6.37575758])

In [98]:
a=pd.read_csv('C:/Users/tjoeun/Desktop/송수린/workspace/weather_project/data/예측용기상대기정보.csv')
b=pd.read_csv('C:/Users/tjoeun/Desktop/송수린/workspace/weather_project/data/기상대기정보.csv')

In [99]:
a.head()

,지점명,일시,지점,기온(°C),강수여부,풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),해면기압(hPa),...,시정(10m),지면온도(°C),PM10,PM2.5,O3,NO2,CO,SO2,월,전날기온
0,서울,2023-02-24 17:00:00,108,7.0,0,3.5,290.0,31.0,1012.8,1023.4,...,2000.0,5.8,34.0,21.0,0.049,0.013,0.4,0.003,2,8.1
1,서울,2023-02-24 18:00:00,108,5.3,0,3.1,290.0,29.0,1013.4,1024.1,...,2000.0,2.6,31.0,20.0,0.048,0.012,0.3,0.003,2,6.3
2,서울,2023-02-24 19:00:00,108,3.8,0,2.7,270.0,34.0,1014.3,1025.0,...,2000.0,0.5,28.0,18.0,0.043,0.015,0.3,0.003,2,4.5
3,서울,2023-02-24 20:00:00,108,2.6,0,3.0,290.0,42.0,1015.3,1026.1,...,2000.0,-0.4,30.0,16.0,0.042,0.013,0.4,0.003,2,3.7
4,서울,2023-02-24 21:00:00,108,1.6,0,3.9,290.0,47.0,1015.6,1026.4,...,2000.0,-0.2,35.0,19.0,0.038,0.015,0.4,0.003,2,3.2


In [100]:
b.head()

,지점명,일시,지점,기온(°C),강수여부,풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),해면기압(hPa),...,Ozon,NO2,CO,SO2,증기압(hPa),이슬점온도(°C),시정(10m),지면온도(°C),월,전날기온
0,철원,2018-08-02 01:00:00,95,28.2,0,1.0,50.0,79.0,987.3,1004.6,...,0.023,0.003,0.3,0.001,30.2,24.2,1522.0,27.7,8,25.0
1,철원,2018-08-02 02:00:00,95,27.3,0,0.7,90.0,85.0,987.5,1004.9,...,0.024,0.003,0.3,0.001,30.7,24.5,1346.0,26.8,8,24.2
2,철원,2018-08-02 03:00:00,95,26.5,0,0.4,0.0,87.0,987.8,1005.2,...,0.024,0.003,0.2,0.001,30.0,24.1,1072.0,26.2,8,23.6
3,철원,2018-08-02 04:00:00,95,26.1,0,1.0,70.0,91.0,987.9,1005.4,...,0.024,0.002,0.2,0.001,30.7,24.5,1125.0,25.7,8,23.7
4,철원,2018-08-02 05:00:00,95,26.5,0,0.6,140.0,90.0,988.4,1005.8,...,0.021,0.002,0.2,0.001,31.1,24.7,1329.0,25.4,8,22.9
